# Thanks for checking out my little side project, analyzing physics teaching at UCSD.

## This notebook focuses on data procurement and cleaning.

## Dataset: student evaluations of 1025 physics courses at UCSD, **scraped** from web (using BeautifulSoup and urllib2).
### Features: 
- instructor (categorical)
- course code (categorical, e.g. 1A for "physics 1A", which is offered every academic quarter)
- year
- term (categorical: Fall,Winter, Spring, Summer)
- lower division, upper division, or lab? (categorical)
- class size
- \# evaluations returned (~ attendance)
- \# hours spent studying / week
- expected grade
- attendance (actually, # evaluations returned / enrollment)

### Labels
- % students recommending course
- % students recommending instructor

### Objectives of this project:
- *describe* this data
- *analyze* predictors of positive outcomes (high % students recommending class and/or instructor)

## dataURL is the web address of the data that we're about to scrape.  
- Originally, the HTML file came from manually querying http://cape.ucsd.edu/responses/Results.aspx for all physics courses
- Ideally, I would like to automate this query, but I do not know how to program Python to interact with search fields... non-trivial
- For now, I've stashed the HTML containing query results on github...

In [1]:
import pandas as pd
import numpy as np
dataURL = 'https://raw.githubusercontent.com/cscottwylie/ucsdCourseEvaluations/master/allPhysicsCapeSummary.html'

# Obtain data from cape.ucsd.edu/responses

### To write this scraper, I had to locate desired data within the html file, using "inspect element"
1. Item called "tbody" contains whole data table
2. Each row corresponds to a specific course, stored in "tr" items
3. Each data point (cell) stored in "td" items

### Put data points in 2D list

### To each row, append hyperlink to detailed course data (future analysis)


In [2]:
def scrapeWeb(dataLocation):   
    
    '''input: URL containing data to be scraped
       output: 2D list (# courses by # features) containing course evaluation data scraped from web'''
    from bs4 import BeautifulSoup
    import urllib2
    page = urllib2.urlopen(dataLocation).read()
    soup = BeautifulSoup(page,"lxml")
    tableDat = soup.find("tbody")                   # tbody is the tag of the whole table
    listTable = [[td.text for td in tr.findAll("td")] 
                 for tr in tableDat.findAll("tr")]       #tr = row, td = data point
    
    # now put links to detailed stats at the end of each row.  
    # Look out for key errors
    def graceful_href_extract(tr):
        try:
             return tr.a["href"]
        except KeyError:
             return ""
            
    trList=tableDat.findAll("tr")
    for i in range(0,len(trList)):
        listTable[i].append(graceful_href_extract(trList[i]))
        
    return listTable

# Our dataframe will be called df.  More columns will be added a bit later.

In [3]:
colNames= ['Instructor', 'Course','Term','Enroll','Evals',
               'Rcmnd_Class','Rcmnd_Instructor','Study_Hrs_Per_Wk',
               'Grade_Expected','Grade', 'Link']
df = pd.DataFrame(scrapeWeb(dataURL), columns=colNames)

In [4]:
def triage169and170(df):
    df.drop(df.index[169:171],inplace=True)
    df.reset_index(drop=True,inplace=True)
    return df
df = triage169and170(df)

In [5]:
def dropLetterGrade(df):
    import re
    df.Grade_Expected = df.Grade_Expected.apply(lambda x: re.sub("[^\d\.]","",str(x)))
    df.Grade = df.Grade.apply(lambda x: re.sub("[^\d\.]","",str(x)))
    return df
df = dropLetterGrade(df)

In [6]:
def removeGarbageChars(df):
    df = df.applymap(lambda x: str(x).replace('%','')
                            .replace('(','')
                            .replace(')','')
                            .replace('\n',''))
    return df
df = removeGarbageChars(df)

In [7]:
def splitCourseIntoCodeAndTitle(df):
    df['Course_Code'] = df['Course'].apply(lambda x: x.split()[1])
    df['Course_Title'] = df.Course.apply(lambda x: ' '.join(x.split()[3:-1]))
    df.drop('Course',inplace=True, axis=1)
    return df

df = splitCourseIntoCodeAndTitle(df)

In [8]:
def appendCourseType(df):
    import re
    def toc(row):
        if type(row.Course_Code) != str:
            return None
        elif re.search('[Ll]', row.Course_Code):
            return 'Lab'
        elif len(re.search('\d+', row.Course_Code).group()) > 1:
            return 'Upper_Division'
        else:
            return 'Lower_Division'
    df['Course_Type'] = df.apply(toc, axis=1) 
    return df
df = appendCourseType(df)

In [9]:
def splitTermIntoYearAndQuarter(df):
    df['Quarter'] = df.Term.str.extract('(\D+)')
    df['Year']    = df.Term.str.extract('(..$)')
    # Now reformat these new columns just a bit
    df.Year = df.Year.apply(lambda x: '20'+x)
    def formatQuarter(s):
        d = dict(S='Summer', FA='Fall', SP='Spring', WI='Winter',SU='Summer')
        return d[s]
    df.Quarter = df.Quarter.apply(formatQuarter)
    df.drop('Term',inplace=True, axis=1)
    return df
df = splitTermIntoYearAndQuarter(df)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1023 entries, 0 to 1022
Data columns (total 14 columns):
Instructor          1023 non-null object
Enroll              1023 non-null object
Evals               1023 non-null object
Rcmnd_Class         1023 non-null object
Rcmnd_Instructor    1023 non-null object
Study_Hrs_Per_Wk    1023 non-null object
Grade_Expected      1023 non-null object
Grade               1023 non-null object
Link                1023 non-null object
Course_Code         1023 non-null object
Course_Title        1023 non-null object
Course_Type         1023 non-null object
Quarter             1023 non-null object
Year                1023 non-null object
dtypes: object(14)
memory usage: 119.9+ KB


In [11]:
def castColumns(df):
    ## These two little functions will facilitate type conversion
    def graceful_float(x):
        try:
            return float(x)
        except ValueError:   # float('N/A') throws this error 
            return None     # pandas sees None as null    
    def graceful_int(x):
        try:
            return int(x)
        except ValueError:   # int('N/A') throws this error 
            return None     # pandas sees None as null
    
    floatCols = ['Rcmnd_Class', 'Rcmnd_Instructor', 'Study_Hrs_Per_Wk', 'Grade_Expected', 'Grade']
    intCols = ['Enroll', 'Evals', 'Year']
    df[floatCols] = df[floatCols].applymap(graceful_float)
    df[intCols] = df[intCols].applymap(graceful_int)
    return df
temp = castColumns(df)

## Let's trade "Evals" for Attendance = Evals/Enroll

In [12]:
df['Attendance'] = df.Evals.astype(float) / df.Enroll
df.drop('Evals', axis=1, inplace=True)

In [13]:
df.Attendance.max()

1.75

In [14]:
df.Attendance[df.Attendance>1] = 0.

/Users/cwylie/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [15]:
df.Attendance.max()

1.0

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1023 entries, 0 to 1022
Data columns (total 14 columns):
Instructor          1023 non-null object
Enroll              1023 non-null int64
Rcmnd_Class         1022 non-null float64
Rcmnd_Instructor    1022 non-null float64
Study_Hrs_Per_Wk    1022 non-null float64
Grade_Expected      1010 non-null float64
Grade               786 non-null float64
Link                1023 non-null object
Course_Code         1023 non-null object
Course_Title        1023 non-null object
Course_Type         1023 non-null object
Quarter             1023 non-null object
Year                1023 non-null int64
Attendance          1023 non-null float64
dtypes: float64(6), int64(2), object(6)
memory usage: 119.9+ KB


In [17]:
df[pd.isnull(df.Rcmnd_Class)]

,Instructor,Enroll,Rcmnd_Class,Rcmnd_Instructor,Study_Hrs_Per_Wk,Grade_Expected,Grade,Link,Course_Code,Course_Title,Course_Type,Quarter,Year,Attendance
362,"Anderson, Michael G.",24,NaN,NaN,NaN,NaN,3.13,https://cape.ucsd.edu/scripts/detailedStats.as...,1AL,Mechanics Laboratory,Lab,Summer,2012,0.041667


### That row is worthless, so out it goes

In [18]:
df = df[pd.notnull(df.Rcmnd_Class)]
df.reset_index(drop=True, inplace=True)

### Of the remaining 12 rows missing Grade_Expected all are also missing Grade.  However, there's still valuable information here, so I'll hold onto these rows for now

In [19]:
df[df.isnull().Grade_Expected]

,Instructor,Enroll,Rcmnd_Class,Rcmnd_Instructor,Study_Hrs_Per_Wk,Grade_Expected,Grade,Link,Course_Code,Course_Title,Course_Type,Quarter,Year,Attendance
62,"Manohar, Aneesh Vasant",1,100.0,100,0.0,NaN,NaN,https://cape.ucsd.edu/responses/CAPEReport.asp...,140B,Statistical & Thermal Phys II,Upper_Division,Winter,2015,1.000000
88,"Fuller, George Michael",19,100.0,100,0.5,NaN,NaN,https://cape.ucsd.edu/responses/CAPEReport.asp...,87,Freshman Seminar,Upper_Division,Winter,2015,0.684211
98,"Manohar, Aneesh Vasant",1,100.0,100,0.0,NaN,NaN,https://cape.ucsd.edu/responses/CAPEReport.asp...,140A,Statistical & Thermal Phys I,Upper_Division,Fall,2014,1.000000
101,"Murphy, Thomas",28,100.0,100,0.5,NaN,NaN,https://cape.ucsd.edu/responses/CAPEReport.asp...,191,Undergraduate Seminar/Physics,Upper_Division,Fall,2014,0.357143
116,"Murphy, Thomas",1,100.0,100,0.0,NaN,NaN,https://cape.ucsd.edu/responses/CAPEReport.asp...,2D,Phys-Relativity&Quantm Physics,Lower_Division,Fall,2014,1.000000
151,"Paar, Hans Peter",5,100.0,100,2.5,NaN,NaN,https://cape.ucsd.edu/responses/CAPEReport.asp...,192,Senior Seminar in Physics,Upper_Division,Spring,2014,0.200000
172,"Burgasser, Adam Jonathan",5,100.0,100,0.5,NaN,NaN,https://cape.ucsd.edu/responses/CAPEReport.asp...,87,Freshman Seminar,Upper_Division,Spring,2014,0.400000
271,"Jun, Suckjoon",1,0.0,0,0.0,NaN,NaN,https://cape.ucsd.edu/responses/CAPEReport.asp...,177,Physics of the Cell,Upper_Division,Spring,2013,1.000000
305,"Hwa, Terence T.",2,100.0,100,0.0,NaN,NaN,https://cape.ucsd.edu/responses/CAPEReport.asp...,176,Quantitative Molecular Biology,Upper_Division,Winter,2013,0.500000
777,"Dubin, Daniel Herschel",17,100.0,100,0.5,NaN,NaN,https://cape.ucsd.edu/scripts/detailedStats.as...,87,Freshman Seminar,Upper_Division,Winter,2010,0.529412


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1022 entries, 0 to 1021
Data columns (total 14 columns):
Instructor          1022 non-null object
Enroll              1022 non-null int64
Rcmnd_Class         1022 non-null float64
Rcmnd_Instructor    1022 non-null float64
Study_Hrs_Per_Wk    1022 non-null float64
Grade_Expected      1010 non-null float64
Grade               785 non-null float64
Link                1022 non-null object
Course_Code         1022 non-null object
Course_Title        1022 non-null object
Course_Type         1022 non-null object
Quarter             1022 non-null object
Year                1022 non-null int64
Attendance          1022 non-null float64
dtypes: float64(6), int64(2), object(6)
memory usage: 119.8+ KB


In [21]:
df.to_csv('cleanedDF.csv')

In [25]:
!ls -l

total 9464
-rw-r-----@ 1 cwylie  501  2077252 Nov 18 14:54 allPhysicsCapeSummary.html
-rw-r--r--  1 cwylie  501    70383 Feb 26 11:19 capeDemo.ipynb
-rw-r--r--  1 cwylie  501   262570 Mar  2 20:32 cleanDFwithPCA.csv
-rw-r--r--  1 cwylie  501   196681 Mar  2 19:56 cleanedDF.csv
-rw-r--r--  1 cwylie  501   355899 Mar  2 21:14 courseEvaluationsEDA-Copy1.ipynb
-rw-r--r--  1 cwylie  501  1661938 Mar  2 21:08 courseEvaluationsEDA.ipynb
-rw-r--r--  1 cwylie  501    30588 Mar  2 21:13 courseEvaluationsScrapeAndClean.ipynb
-rw-r--r--  1 cwylie  501   171170 Nov 18 20:09 physicsSummary.csv


In [26]:
!pwd

/Users/cwylie/Documents/ipythonPracOct2015/insightDemo/cape


In [ ]:
!mkdir ../tempHide
!mv 